In [1]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
os.chdir('../')
from utils.utils import *
from models.Transformer.layers import Transformer

In [2]:
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8
jpn_vocab_size = 8000
en_vocab_size = 8000

jpn_txt_path = "dataset/jpn_data.txt"
en_txt_path = "dataset/en_data.txt"
ckpt_path = "models/Transformer/ckpt/"
log_path = "logs/"

In [3]:
jpn_data = get_data(jpn_txt_path)
en_data = get_data(en_txt_path)
JPN_MAX_LEN = get_max_len(jpn_data)
EN_MAX_LEN = get_max_len(en_data)
# include [BOS] and [EOS] to each max len above
JPN_MAX_LEN += 2
EN_MAX_LEN += 2

transformer = Transformer(num_layers=num_layers,
                          d_model=d_model,
                          num_heads=num_heads,
                          dff=dff,
                          input_vocab_size=jpn_vocab_size,
                          target_vocab_size=en_vocab_size,
                          pe_input=JPN_MAX_LEN,
                          pe_target=EN_MAX_LEN)

learning_rate = CustomSchedule(d_model)
optimizer = Adam(learning_rate,
                 beta_1=0.9,
                 beta_2=0.98,
                 epsilon=1e-9)

In [20]:
#latest = tf.train.latest_checkpoint(ckpt_path)
latest = os.path.join(ckpt_path, "ckpt-10")
checkpoint = tf.train.Checkpoint(transformer=transformer,
                                 optimizer=optimizer)
checkpoint.restore(latest)

In [21]:
checkpoint.transformer

In [38]:
translate("お腹が空いています。", EN_MAX_LEN, checkpoint.transformer, result_log=True, plot="")
translate("今日は楽しい日ですね。", EN_MAX_LEN, checkpoint.transformer, result_log=True, plot="")
translate("外国に行ってみたいです。", EN_MAX_LEN, checkpoint.transformer, result_log=True, plot="")

Input: お腹が空いています。
Predicted translation: I'm hungry.
Input: 今日は楽しい日ですね。
Predicted translation: It is a pleasant day today.
Input: 外国に行ってみたいです。
Predicted translation: I'm looking forward to going abroad.


"I'm looking forward to going abroad."